In [6]:
import findspark
findspark.init()
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import functions as f

spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "10g") \
    .appName('movieRecommenderPySpark') \
    .getOrCreate()

sc = spark.sparkContext
sc.setCheckpointDir('checkpoint')

data_type = "small"

In [17]:
ratings = (
    spark.read.csv(
        path=f"../data/{data_type}/ratings.csv",
        sep=",", header=True, quote='"', schema="userId INT, movieId INT, rating DOUBLE, timestamp INT",
    ).select("userId", "movieId", "rating")
    .cache()
)

rating= ratings.filter(f.col('userId')==1).rdd.map(lambda row: {'userId':row[0],'movieId':row[1],'rating':row[2]}).collect()
Lawrence_movie_ratings = pd.DataFrame(rating)
Lawrence_movie_ratings.drop(['userId'], axis=1, inplace=True)
Lawrence_movie_ratings

,movieId,rating
0,1,4.0
1,3,4.0
2,6,4.0
3,47,5.0
4,50,5.0
...,...,...
227,3744,4.0
228,3793,5.0
229,3809,4.0
230,4006,4.0


In [18]:
missing_values = ['na','--','?','-','None','none','non']
movies_df = pd.read_csv('../data/small/movies.csv', na_values=missing_values)
movies_df['genres'] = movies_df.genres.str.split('|')

# First let's make a copy of the movies_df
movies_with_genres = movies_df.copy(deep=True)



# Let's iterate through movies_df, then append the movie genres as columns of 1s or 0s.
# 1 if that column contains movies in the genre at the present index and 0 if not.

x = []
for index, row in movies_df.iterrows():
    x.append(index)
    for genre in row['genres']:
        movies_with_genres.at[index, genre] = 1

# Confirm that every row has been iterated and acted upon
print(len(x) == len(movies_df))

movies_with_genres.head(3)


#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
movies_with_genres = movies_with_genres.fillna(0)
Lawrence_genres_df = movies_with_genres[movies_with_genres.movieId.isin(Lawrence_movie_ratings.movieId)]
Lawrence_genres_df.drop(['movieId','title','genres'], axis=1, inplace=True)
Lawrence_genres_df.head(5)

True


C:\Users\Dat\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,...,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed),Science Fiction
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
print(Lawrence_movie_ratings.rating)

0      4.0
1      4.0
2      4.0
3      5.0
4      5.0
      ... 
227    4.0
228    5.0
229    4.0
230    4.0
231    5.0
Name: rating, Length: 232, dtype: float64


In [22]:
print('Shape of Lawrence_movie_ratings is:',Lawrence_movie_ratings.shape)
print('Shape of Lawrence_genres_df is:',Lawrence_genres_df.shape)

Shape of Lawrence_movie_ratings is: (232, 2)
Shape of Lawrence_genres_df is: (232, 21)


In [21]:
Lawrence_profile = Lawrence_genres_df.T.dot(Lawrence_movie_ratings.rating)

ValueError: matrices are not aligned